In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# --- Helper Function ---
def read_excel_file(file_path, sheet_name=None, columns=None, rename_columns=None, dropna_cols=None):
    try:
        excel_file = pd.ExcelFile(file_path)
        if isinstance(sheet_name, list):
            data = {}
            for sheet in sheet_name:
                if sheet not in excel_file.sheet_names and not isinstance(sheet, int):
                    raise ValueError(f"Sheet '{sheet}' not found in {file_path}")
                df = excel_file.parse(sheet)
                if columns:
                    missing_cols = [col for col in columns if col not in df.columns]
                    if missing_cols:
                        raise ValueError(f"Missing columns in sheet '{sheet}': {missing_cols}")
                    df = df[columns]
                if rename_columns:
                    df = df.rename(columns=rename_columns)
                if dropna_cols:
                    df = df.dropna(subset=dropna_cols)
                data[sheet] = df
            return data
        if sheet_name is None:
            sheet_name = excel_file.sheet_names[0]
        if sheet_name not in excel_file.sheet_names and not isinstance(sheet_name, int):
            raise ValueError(f"Sheet '{sheet_name}' not found in {file_path}")
        df = excel_file.parse(sheet_name)
        if columns:
            missing_cols = [col for col in columns if col not in df.columns]
            if missing_cols:
                raise ValueError(f"Missing columns in sheet '{sheet_name}': {missing_cols}")
            df = df[columns]
        if rename_columns:
            df = df.rename(columns=rename_columns)
        if dropna_cols:
            df = df.dropna(subset=dropna_cols)
        return df
    except FileNotFoundError:
        raise FileNotFoundError(f"File not found: {file_path}")
    except Exception as e:
        raise Exception(f"Error reading Excel file: {str(e)}")

# --- Load Data ---
file_paths = {
    'cross_cutting': 'cleaned_bulk_data_cross_cutting.xlsx',
    'food': 'cleaned_bulk_data_food.xlsx',
    'energy': 'cleaned_bulk_data_energy.xlsx',
    'finance': 'cleaned_bulk_data_finance.xlsx'
}

# Load metadata
metadata_df = read_excel_file(
    file_paths['cross_cutting'],
    sheet_name="Country ID and group",
    columns=['geography_code', 'countries', 'developing_region_un', 'development_status'],
    rename_columns={'developing_region_un': 'developing_region'},
    dropna_cols=['geography_code', 'countries', 'development_status']
)
metadata_df['developing_region'] = metadata_df['developing_region'].replace({
    'Developed - Asia and Oceania': 'Asia and Oceania',
    'Developed - America': 'America'
})
metadata_df = metadata_df[metadata_df['geography_code'] != 'developed'].drop_duplicates(subset='geography_code')
print(f"Metadata shape: {metadata_df.shape}")

# Load indicators
indicators = {
    'inflation_rate': read_excel_file(
        file_paths['cross_cutting'], sheet_name="Inflation rates",
        columns=['date', 'geography', 'value'], dropna_cols=['geography', 'value']
    ).rename(columns={'value': 'inflation_rate'}),
    'fao_food_index': read_excel_file(
        file_paths['food'], sheet_name="FAO Food Price Index",
        columns=['date', 'value'], dropna_cols=['value']
    ).rename(columns={'value': 'fao_food_index'}),
    'food_inflation_rate': read_excel_file(
        file_paths['food'], sheet_name="Food price inflation",
        columns=['date', 'geography', 'value'], dropna_cols=['geography', 'value']
    ).rename(columns={'value': 'food_inflation_rate'}),
    'oil_price': read_excel_file(
        file_paths['energy'], sheet_name="Oil prices",
        columns=['date', 'oil_brent'], dropna_cols=['oil_brent']
    ).rename(columns={'oil_brent': 'oil_price'}),
    'gas_price': read_excel_file(
        file_paths['energy'], sheet_name="Gas prices",
        columns=['date', 'gas_ttf'], dropna_cols=['gas_ttf']
    ).rename(columns={'gas_ttf': 'gas_price'})
}

# Merge indicators
merged_df = None
for key, df in indicators.items():
    df['date'] = pd.to_datetime(df['date'])
    print(f"{key} initial shape: {df.shape}")
    if key in ['fao_food_index', 'oil_price', 'gas_price']:
        geographies = metadata_df['geography_code'].unique()
        df = df.assign(key=1).merge(
            pd.DataFrame({'geography': geographies, 'key': 1}),
            on='key'
        ).drop('key', axis=1)
    df[key] = pd.to_numeric(df[key], errors='coerce')
    if key == 'oil_price':
        df = df[(df[key] >= 0) & (df[key] <= 500)]
    elif key == 'gas_price':
        df = df[(df[key] >= 0) & (df[key] <= 100)]
    elif key in ['inflation_rate', 'food_inflation_rate']:
        df = df[(df[key] >= -0.5) & (df[key] <= 50.0)]
    elif key == 'fao_food_index':
        df = df[(df[key] >= 0) & (df[key] <= 500)]
    print(f"{key} after filtering: {df.shape}")
    if merged_df is None:
        merged_df = df[['date', 'geography', key]]
    else:
        merged_df = merged_df.merge(
            df[['date', 'geography', key]], on=['date', 'geography'], how='outer'
        )
    print(f"Merged shape after {key}: {merged_df.shape}")

# Merge with metadata
merged_df = merged_df.merge(
    metadata_df[['geography_code', 'countries', 'development_status', 'developing_region']],
    left_on='geography', right_on='geography_code', how='left'
).drop(columns='geography_code')
print(f"Merged shape after metadata: {merged_df.shape}")

# Impute missing metadata
merged_df['development_status'] = merged_df['development_status'].fillna('Unknown')
merged_df['developing_region'] = merged_df['developing_region'].fillna('Unknown')
merged_df['countries'] = merged_df['countries'].fillna(merged_df['geography'])

# Impute missing numerical values
for col in ['inflation_rate', 'food_inflation_rate', 'fao_food_index', 'oil_price', 'gas_price']:
    merged_df[col] = merged_df.groupby('geography')[col].ffill().fillna(merged_df[col].median())

# Feature engineering
merged_df = merged_df.sort_values(['geography', 'date'])
for col in ['inflation_rate', 'food_inflation_rate', 'fao_food_index', 'oil_price', 'gas_price']:
    merged_df[f'{col}_lag1'] = merged_df.groupby('geography')[col].shift(1)
merged_df['oil_price_lag12'] = merged_df.groupby('geography')['oil_price'].shift(12)

# Compute z-scores for key indicators
for col in ['inflation_rate', 'food_inflation_rate', 'oil_price']:
    mean = merged_df.groupby('geography')[col].transform('mean')
    std = merged_df.groupby('geography')[col].transform('std')
    merged_df[f'{col}_z'] = (merged_df[col] - mean) / std
    merged_df[f'{col}_z'] = merged_df[f'{col}_z'].fillna(0)  # Handle NaN z-scores

# Add conflict risk
high_risk_regions = ['Africa', 'Western Asia']
merged_df['conflict_risk'] = merged_df['developing_region'].isin(high_risk_regions).astype(int)

# --- Statistical Crisis Detection ---
def assign_crisis_type(row):
    if row['inflation_rate_z'] > 2 or row['inflation_rate'] > 0.2:
        return 'Inflationary'
    if (row['food_inflation_rate_z'] > 1.5) and row['development_status'] in ['Developing', 'Unknown']:
        return 'Food Insecurity'
    if pd.notna(row['oil_price_lag12']) and (row['oil_price_z'] > 2 or row['oil_price'] > row['oil_price_lag12'] * 1.5):
        return 'Energy'
    return 'None'

def compute_crisis_prob(row):
    prob = (
        0.4 * max(0, min(1, row['inflation_rate_z'] / 2)) +  # Normalize z-score
        0.4 * max(0, min(1, row['food_inflation_rate_z'] / 1.5)) +
        0.2 * max(0, min(1, row['oil_price_z'] / 2)) +
        0.1 * row['conflict_risk']  # Small boost for high-risk regions
    )
    return min(prob, 1.0)

merged_df['crisis_type'] = merged_df.apply(assign_crisis_type, axis=1)
merged_df['crisis_probability'] = merged_df.apply(compute_crisis_prob, axis=1)
merged_df['crisis'] = (merged_df['crisis_type'] != 'None').astype(int)

# --- Future Predictions ---
future_dates = pd.date_range('2025-06-01', '2025-12-01', freq='MS')
future_data = []
required_cols = [
    'inflation_rate', 'food_inflation_rate', 'fao_food_index', 'oil_price', 'gas_price',
    'inflation_rate_lag1', 'food_inflation_rate_lag1', 'fao_food_index_lag1',
    'oil_price_lag1', 'gas_price_lag1', 'oil_price_lag12',
    'conflict_risk', 'development_status', 'developing_region', 'countries'
]
for geo in merged_df['geography'].unique():
    last_data = merged_df[merged_df['geography'] == geo].iloc[-1][required_cols]
    for date in future_dates:
        row = last_data.copy()
        row['date'] = date
        row['geography'] = geo
        future_data.append(row)
future_df = pd.DataFrame(future_data)

# Impute missing values in future_df
for col in [
    'inflation_rate', 'food_inflation_rate', 'fao_food_index', 'oil_price', 'gas_price',
    'inflation_rate_lag1', 'food_inflation_rate_lag1', 'fao_food_index_lag1',
    'oil_price_lag1', 'gas_price_lag1', 'oil_price_lag12'
]:
    future_df[col] = future_df[col].fillna(future_df[col].median())

# Compute z-scores (use historical means/stds from merged_df)
for col in ['inflation_rate', 'food_inflation_rate', 'oil_price']:
    mean = merged_df.groupby('geography')[col].mean().reindex(future_df['geography']).values
    std = merged_df.groupby('geography')[col].std().reindex(future_df['geography']).values
    future_df[f'{col}_z'] = (future_df[col] - mean) / std
    future_df[f'{col}_z'] = future_df[f'{col}_z'].fillna(0)

# Apply statistical crisis detection
future_df['crisis_type'] = future_df.apply(assign_crisis_type, axis=1)
future_df['crisis_probability'] = future_df.apply(compute_crisis_prob, axis=1)
future_df['crisis_predicted'] = (future_df['crisis_probability'] > 0.5).astype(int)

# Select output columns
output_df = future_df[[
    'date', 'geography', 'countries', 'development_status', 'developing_region',
    'crisis_probability', 'crisis_predicted', 'crisis_type'
]]

# Export
output_df.to_csv('crisis_predictions.csv', index=False)
print("✅ Predictions exported to 'crisis_predictions.csv'")

Metadata shape: (188, 4)
inflation_rate initial shape: (13554, 3)
inflation_rate after filtering: (13464, 3)
Merged shape after inflation_rate: (13464, 3)
fao_food_index initial shape: (422, 2)
fao_food_index after filtering: (79336, 3)
Merged shape after fao_food_index: (80014, 4)
food_inflation_rate initial shape: (12687, 3)
food_inflation_rate after filtering: (12453, 3)
Merged shape after food_inflation_rate: (80104, 5)
oil_price initial shape: (2276, 2)
oil_price after filtering: (427888, 3)
Merged shape after oil_price: (493906, 6)
gas_price initial shape: (2276, 2)
gas_price after filtering: (157168, 3)
Merged shape after gas_price: (493906, 7)
Merged shape after metadata: (493906, 10)
✅ Predictions exported to 'crisis_predictions.csv'


In [2]:
import pandas as pd
import numpy as np

# Assuming merged_df and metadata_df are already created
def compute_crisis_prob(row):
    prob = (
        0.4 * row['inflation_rate_z'] / 2 +
        0.4 * row['food_inflation_rate_z'] / 1.5 +
        0.2 * row['oil_price_z'] / 2 +
        0.1 * row.get('conflict_risk', 0)
    )
    return min(prob, 1.0)

def assign_crisis_type(row):
    if row['crisis_probability'] > 0.5:
        if row['inflation_rate_z'] > 1.5 or row['inflation_rate'] > 0.15:
            return 'Inflationary'
        if (row['food_inflation_rate_z'] > 1.0) and row['development_status'] in ['Developing', 'Unknown']:
            return 'Food Insecurity'
        if pd.notna(row['oil_price_lag12']) and (row['oil_price_z'] > 1.5 or row['oil_price'] > row['oil_price_lag12'] * 1.3):
            return 'Energy'
        return 'Unknown Crisis'
    return 'None'

# Future Predictions
future_dates = pd.date_range('2025-06-01', '2025-12-01', freq='MS')
valid_geographies = metadata_df[metadata_df['development_status'].isin(['Developed', 'Developing'])]['geography_code'].unique()
future_data = []
required_cols = [
    'inflation_rate', 'food_inflation_rate', 'fao_food_index', 'oil_price', 'gas_price',
    'inflation_rate_lag1', 'food_inflation_rate_lag1', 'fao_food_index_lag1',
    'oil_price_lag1', 'gas_price_lag1', 'oil_price_lag12',
    'conflict_risk', 'development_status', 'developing_region', 'countries'
]

for geo in merged_df[merged_df['geography'].isin(valid_geographies)]['geography'].unique():
    geo_data = merged_df[merged_df['geography'] == geo].sort_values('date')
    last_data = geo_data.iloc[-1][required_cols].copy()
    for col in ['inflation_rate', 'food_inflation_rate', 'oil_price']:
        last_data[col] = geo_data[col].tail(3).mean() if len(geo_data) >= 3 else geo_data[col].mean()
    for date in future_dates:
        row = last_data.copy()
        row['date'] = date
        row['geography'] = geo
        future_data.append(row)
future_df = pd.DataFrame(future_data)

# Impute missing values
for col in [
    'inflation_rate', 'food_inflation_rate', 'fao_food_index', 'oil_price', 'gas_price',
    'inflation_rate_lag1', 'food_inflation_rate_lag1', 'fao_food_index_lag1',
    'oil_price_lag1', 'gas_price_lag1', 'oil_price_lag12'
]:
    future_df[col] = future_df[col].fillna(future_df[col].median())

# Debug input data
print("Last row of merged_df:")
print(merged_df.groupby('geography').last()[['inflation_rate', 'food_inflation_rate', 'oil_price']].head(10))
print("Food inflation stds:")
print(merged_df.groupby('geography')['food_inflation_rate'].std().describe())

# Compute z-scores
for col in ['inflation_rate', 'food_inflation_rate', 'oil_price']:
    if col == 'oil_price':
        mean = merged_df[col].mean()
        std = merged_df[col].std() if merged_df[col].std() > 0 else 1.0
        future_df[f'{col}_z'] = (future_df[col] - mean) / std
    else:
        mean = merged_df.groupby('geography')[col].mean().reindex(future_df['geography']).values
        std = merged_df.groupby('geography')[col].std().reindex(future_df['geography']).values
        global_std = merged_df[col].std() if merged_df[col].std() > 0 else 1.0
        std = np.where(std == 0, global_std, std)
        future_df[f'{col}_z'] = (future_df[col] - mean) / std
    future_df[f'{col}_z'] = future_df[f'{col}_z'].fillna(0)

# Debug z-scores
print("Z-score summary:")
print(future_df[['geography', 'inflation_rate_z', 'food_inflation_rate_z', 'oil_price_z']].describe())
print("Z-scores for high-probability countries:")
high_prob_geos = ['CHE', 'JOR', 'OMN', 'PAN']
print(future_df[future_df['geography'].isin(high_prob_geos)][['geography', 'inflation_rate', 'inflation_rate_z', 'food_inflation_rate_z', 'oil_price_z', 'oil_price', 'oil_price_lag12']].drop_duplicates())

# Apply crisis detection
future_df['crisis_probability'] = future_df.apply(compute_crisis_prob, axis=1)
future_df['crisis_type'] = future_df.apply(assign_crisis_type, axis=1)
future_df['crisis_predicted'] = (future_df['crisis_probability'] > 0.5).astype(int)

# Select output columns
output_df = future_df[[
      'geography', 'countries', 'development_status', 'developing_region',
    'crisis_probability', 'crisis_predicted', 'crisis_type'
]]

# Debug output
print(f"Output rows: {len(output_df)}")
print(f"Unique geographies: {len(output_df['geography'].unique())}")
 

Last row of merged_df:
           inflation_rate  food_inflation_rate   oil_price
geography                                                 
AFG                 0.038                 0.05  140.534437
AGO                 0.038                 0.05  140.534437
ALB                 0.038                 0.05  140.534437
ARE                 0.038                 0.05  140.534437
ARG                 0.038                 0.05  140.534437
ARM                 0.038                 0.05  140.534437
ATG                 0.038                 0.05  140.534437
AUS                 0.038                 0.05  140.534437
AUT                 0.038                 0.05  140.534437
AZE                 0.038                 0.05  140.534437
Food inflation stds:
count    195.000000
mean       0.191262
std        1.004724
min        0.000000
25%        0.000000
50%        0.035279
75%        0.056234
max        9.807441
Name: food_inflation_rate, dtype: float64
Z-score summary:
       inflation_rate_z  food

In [3]:
# Drop duplicates based on geography, crisis_probability, and crisis_type
unique_crises = output_df.drop_duplicates(subset=['geography', 'crisis_probability', 'crisis_type'])

# Filter for predicted crises only
unique_crises_filtered = unique_crises[unique_crises['crisis_predicted'] == 1][['geography', 'crisis_probability', 'crisis_type']]

# Print unique high-probability crises
print("High-probability crises (unique):")
print(unique_crises_filtered.reset_index(drop=True))

print(unique_crises_filtered)
# Export
output_df.to_csv('crisis_predictions.csv', index=False)
print("✅ Predictions exported to 'crisis_predictions.csv'") 


High-probability crises (unique):
  geography  crisis_probability      crisis_type
0       CHE            0.689696     Inflationary
1       JOR            0.599158  Food Insecurity
2       OMN            0.630480     Inflationary
3       PAN            0.656394  Food Insecurity
       geography  crisis_probability      crisis_type
493747       CHE            0.689696     Inflationary
493802       JOR            0.599158  Food Insecurity
493848       OMN            0.630480     Inflationary
493850       PAN            0.656394  Food Insecurity
✅ Predictions exported to 'crisis_predictions.csv'


In [4]:
print(unique_crises_filtered.reset_index(drop=True))


  geography  crisis_probability      crisis_type
0       CHE            0.689696     Inflationary
1       JOR            0.599158  Food Insecurity
2       OMN            0.630480     Inflationary
3       PAN            0.656394  Food Insecurity
